# Preprossessing & Machine Learning 

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.svm import SVC
import xgboost as xgb # pip install xgboost
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, classification_report, confusion_matrix
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_score
from statsmodels.tsa.arima.model import ARIMA # Pour ARIMA
from prophet import Prophet # pip install prophet

import matplotlib.pyplot as plt
import seaborn as sns

# Pour la reproductibilité
np.random.seed(42)

In [2]:
#load data 
df_global=pd.read_csv("../code/achat_prod_fournisseur_stock.csv") #upload from code file

In [3]:
df_global.describe()
df_global["date_achat"]=pd.to_datetime(df_global["date_achat"])
df_global.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   id_achat               10000 non-null  object        
 1   date_achat             10000 non-null  datetime64[ns]
 2   id_produit             10000 non-null  object        
 3   quantité               10000 non-null  int64         
 4   id_fournisseur         10000 non-null  object        
 5   prix_unitaire          10000 non-null  float64       
 6   délai_livraison_jours  10000 non-null  int64         
 7   montant_total          10000 non-null  float64       
 8   mois                   10000 non-null  int64         
 9   année                  10000 non-null  int64         
 10  jour_semaine           10000 non-null  int64         
 11  catégorie              10000 non-null  object        
 12  marque                 10000 non-null  object        
 13  pr

In [4]:
# Génération de la variable cible 'niveau_stock' (catégorielle)
bins = [0, 1.5, 3, float('inf')]
labels = ['bas', 'moyen', 'élevé']
# Ratio stock actuel (quantité) par rapport au stock minimum
# (simplification, dans la réalité le stock actuel serait une autre colonne)
df_global['ratio_stock'] = df_global['niveau_stock'] / df_global['stock_minimum']
df_global['niveau_stock_cat'] = pd.cut(df_global['ratio_stock'], bins=bins, labels=labels, right=False)
# Génération de la variable cible 'rupture' (booléen)
# Si niveau_stock est 'bas' et délai de livraison élevé -> risque de rupture
df_global['rupture'] = ((df_global['niveau_stock_cat'] == 'bas') & (df_global['délai_livraison_jours'] > 7))

***explication des modifications apportées***
- Changement des bornes des catégories de niveau de stock pour mieux refléter la distribution des données.
- Activation de la création de la colonne 'niveau_stock_cat' pour catégoriser le niveau de stock.
- Activation de la création de la colonne 'rupture' pour indiquer les ruptures de stock potentielles.
    

In [5]:
# Pour la segmentation fournisseur, ajoutons 'taille_fournisseur'
taille_fourn = df_global.groupby('id_fournisseur')['id_produit'].nunique().rename('taille_fournisseur_nb_produits')
df_global = df_global.merge(taille_fourn, on='id_fournisseur', how='left')

In [6]:
# Pour la classification de fiabilité fournisseur, créons une catégorie
# Si fiabilité (score continu) > 0.85 -> fiable, sinon -> moins_fiable
df_global['fiabilité_catégorie'] = pd.cut(df_global['fiabilité'], bins=[0, 0.85, 1.01], labels=['moins_fiable', 'fiable'], right=False)

In [7]:
# --- Fonctions Utilitaires (pour éviter la répétition) ---
def get_preprocessor(numerical_features, categorical_features):
    """Crée un ColumnTransformer pour le prétraitement."""
    numerical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler())
    ])
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
    ])
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_transformer, numerical_features),
            ('cat', categorical_transformer, categorical_features)
        ], remainder='passthrough' # pour les colonnes non spécifiées (si besoin)
    )
    return preprocessor


In [8]:
# --- Tâche 1: Régression du montant total ---
print("\n--- Tâche 1: Régression du montant_total ---")
df_task1 = df_global.copy()
X1_cols_num = ['quantité', 'prix_unitaire', 'délai_livraison_jours']
X1_cols_cat = ['id_produit', 'catégorie', 'marque', 'id_fournisseur', 'ville', 'pays']
y1_col = 'montant_total'


--- Tâche 1: Régression du montant_total ---


In [9]:
X1 = df_task1[X1_cols_num + X1_cols_cat]
y1 = df_task1[y1_col]

# Séparation des données
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.2, random_state=42)

# Prétraitement
preprocessor1 = get_preprocessor(X1_cols_num, X1_cols_cat)


In [10]:
# Modèles
models1 = {
    "Régression Linéaire": LinearRegression(),
    "Random Forest Regressor": RandomForestRegressor(random_state=42),
    "XGBoost Regressor": xgb.XGBRegressor(objective='reg:squarederror', random_state=42)
}


In [11]:
for name, model in models1.items():
    print(f"\nEntraînement du modèle: {name}")
    pipeline = Pipeline(steps=[('preprocessor', preprocessor1), ('regressor', model)])
    pipeline.fit(X1_train, y1_train)
    y1_pred = pipeline.predict(X1_test)
    
    mse = mean_squared_error(y1_test, y1_pred)
    r2 = r2_score(y1_test, y1_pred)
    print(f"Résultats pour {name}:")
    print(f"  Mean Squared Error: {mse:.2f}")
    print(f"  R² Score: {r2:.2f}")



Entraînement du modèle: Régression Linéaire
Résultats pour Régression Linéaire:
  Mean Squared Error: 82158977.18
  R² Score: 0.79

Entraînement du modèle: Random Forest Regressor
Résultats pour Random Forest Regressor:
  Mean Squared Error: 14347.04
  R² Score: 1.00

Entraînement du modèle: XGBoost Regressor
Résultats pour XGBoost Regressor:
  Mean Squared Error: 158505.45
  R² Score: 1.00


In [12]:
## f